# PREDICTION OF ANXIETY,DEPRESSION AND STRESS

In [1]:
# basic libraries - musts
import numpy as np
import pandas as pd
#impute missings with KNN
from sklearn.impute import KNNImputer
#to convert country codes to alpha 3
!pip install pycountry_convert
from pycountry_convert import country_alpha2_to_country_name, country_name_to_country_alpha3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 5.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 316 kB 42.6 MB/s 
     |████████████████████████████████| 210 kB 65.5 MB/s 
     |████████████████████████████████| 298 kB 2.9 MB/s 
     |████████████████████████████████| 297 kB 35.5 MB/s 
     |████████████████████████████████| 297 kB 52.7 MB/s 
     |████████████████████████████████| 297 kB 3.4 MB/s 
     |████████████████████████████████| 296 kB 36.7 MB/s 
     |████████████████████████████████| 296 kB 58.5 MB/s 
     |████████████████████████████████| 280 kB 52.7 MB/s 
     |████████████████████████████████| 280 kB 59.5 MB/s 
     |████████████████████████████████| 280 kB 74.8 MB/s 
     |████████████████████████████████| 280 kB 69.8 MB/s 
     |█████████████████

In [2]:
import matplotlib.pyplot as plt


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:


!ls

drive  sample_data


In [8]:
dataframe = pd.read_csv('/drive/MyDrive/data.csv', delimiter='\t')



FileNotFoundError: ignored

In [ ]:
dataframe.head()

In [ ]:
dataframe.shape

The DASS is a 42-item self-administered questionnaire designed to measure the magnitude of three negative emotional states: depression, anxiety, and stress. The DASSDepression focuses on reports of low mood, motivation, and self-esteem, DASS-anxiety on physiological arousal, perceived panic, and fear, and DASS-stress on tension and irritability. Instructions to client and scoring: A respondent indicates on a 4-point scale the extent to which each of 42 statements applied over the past week. A printed overlay is used to obtain total scores for each subscale. Higher scores on each subscale indicate increasing severity of depression, anxiety, or stress

The DASS has 3 parts;

    0 - 14 : Depression
    15 - 28 : Anxiety
    29 - 42 : Stress

And these three parts were evaluated in 3 parts also;

    Physical Symptoms
    Mental Symptoms
    Total Score


In [ ]:
#Calculating DASS scores
df = dataframe.copy() # taking a copy for bakup
df.iloc[:,0:126]
#A - Question 1 to 42 are stored in A(eg:Q1A),timetaken in milliseconds to answer the qn is stored in E(Q1E) 
#and the question's position in the survey is stored in I(Q1I) 
#The questionnaired has three subscales(D,A,S) with 14 items in each subscale for the DASS 42 and the max score of 42 item is 126

questions = [i for i in df.iloc[:,0:126] if  'A' in i]
time = [i for i in df.iloc[:,0:126] if  'E' in i] # should be equal to testelapse
position = [i for i in df.iloc[:,0:126] if  'I' in i]

# save items in another dataframe
item_positions = df[position] 
dass = df[questions]
testelapse = df[time]

df.drop(position, axis=1, inplace=True)

In [ ]:
# DASS score calculation
## Q2, Q4, Q7, Q15, Q19, Q23, Q25, Q41 questions are categorized as physical symptoms
# other questions are categorized as mental symptoms
##  _tot = totalqns, _physical_sym_qns = physical symptom, _mental_sym_qns = mental_symptom

df['depression_totalqns'] = np.sum(dass.iloc[:,0:14],axis=1)
df['depression_physical_sym_qns'] = np.sum(dass.loc[:,['Q2A','Q4A','Q7A']],axis=1)
df['depression_mental_sym_qns'] = df['depression_totalqns'] - df['depression_physical_sym_qns']
df['anxiety_totalqns'] = np.sum(dass.iloc[:,15:28],axis=1)
df['anxiety_physical_sym_qns'] = np.sum(dass.loc[:,['Q15A','Q19A','Q23A','Q25A']],axis=1)
df['anxiety_mental_sym_qns'] = df['anxiety_totalqns'] - df['anxiety_physical_sym_qns']
df['stress_totalqns'] = np.sum(dass.iloc[:,29:42],axis=1)
df['stress_physical_sym_qns'] = np.sum(dass.loc[:,['Q41A']],axis=1)
df['stress_mental_sym_qns'] = df['stress_totalqns'] - df['stress_physical_sym_qns']
df['total'] = np.sum(dass.iloc[:,0:42], axis=1)

In [ ]:
#total time is not equal to testelapse.
df['testelapse'] = np.sum(df[time],axis=1) / 1000

In [ ]:
# now drop time related features
df.drop(df[time],axis=1, inplace=True)
df.drop(df[questions],axis=1, inplace= True)

# Ten item Personality Inventory(TIPI)

The Ten-Item Personality Inventory (TIPI) is a brief assessment of the Big Five personality dimensions:
(1) Extraversion, (2) Agreeableness, (3) Conscientiousness, (4) Emotional Stability, and (5) Openness to Experience. 
Items are rated on a scale from 1, disagree strongly, to 7, agree strongly.

TIPI Scoring is done on the basis of this article:https://gosling.psy.utexas.edu/scales-weve-developed/ten-item-personality-measure-tipi/

Scoring the TIPI

1. Recode the reverse-scored items (i.e., recode a 7 with a 1, a 6 with a 2, a 5 with a 3, etc.). The reverse scored items are 2, 4, 6, 8, & 10.

2. Take the AVERAGE of the two items (the standard item and the recoded reverse-scored item) that make up each scale.

Example using the Extraversion scale: A participant has scores of 5 on item 1 (Extraverted, enthusiastic) and and 2 on item 6 (Reserved, quiet). First, recode the reverse-scored item (i.e., item 6), replacing the 2 with a 6. Second, take the average of the score for item 1 and the (recoded) score for item 6. So the TIPI Extraversion scale score would be: (5 + 6)/2 = 5.5

In [ ]:
# TIPI
conversion = {1: 7,
              2: 6,
              3: 5,
              4: 4,
              5: 3,
              6: 2,
              7: 1}

#negative items are 2 ,4, 6, 8, 10
negative_items = ['TIPI2', 'TIPI4', 'TIPI6', 'TIPI8', 'TIPI10']

In [ ]:
print(conversion)

In [ ]:
for x in negative_items:
    df[x] = df[x].map(conversion)

In [ ]:
print(df.columns)

In [ ]:
tipi = df[[i for i in df.columns if 'TIPI' in i]]

df['TIPI_extraversion'] = (df['TIPI1'] + df['TIPI6']) / 2
df['TIPI_agreeableness'] = (df['TIPI2'] + df['TIPI7']) / 2
df['TIPI_conscientiousness'] = (df['TIPI3'] + df['TIPI8']) / 2
df['TIPI_emotional_stability'] = (df['TIPI4'] + df['TIPI9']) / 2
df['TIPI_openness_exp'] = (df['TIPI5'] + df['TIPI10']) / 2

In [ ]:
# drop tipi
df.drop(tipi, axis=1, inplace=True)

VCL

A value of 1 is checked, 0 means unchecked. The words at VCL6, VCL9, and VCL12 are not real words and can be used as a validity check.

In [ ]:
VCL_questions = df[[i for i in df.columns if 'VCL' in i]]

In [ ]:
# renaming VCL's with words
names = ['boat',
         'incoherent', 
         'pallid',
         'robot',
         'audible', 
         'reliability_q1', 
         'paucity',
         'epistemology',
         'reliability_q2',
         'decide',
         'pastiche',
         'reliability_q3',
         'abysmal',
         'lucid',
         'betray',
         'funny']

for x,y in enumerate(names):
    df.rename(columns = {'VCL'+str(x+1) : y}, inplace = True)


for x in names:
    df[x] = df[x].astype('object')

In [ ]:
df.info()

In [ ]:
# change categories to category / object type
categories = ['source', 'uniquenetworklocation', 'screensize', 'country',
'major', 'familysize', 'married', 'voted', 'race',  'orientation', 'religion',
'religion', 'hand', 'gender','engnat', 'urban', 'education']

for column in categories:
    df[column] = df[column].astype('object')

In [ ]:
print(df[column])

In [ ]:
#Simplifying Major column to cluster everything similar and to replace nAn with OTHER

df['major'] = df['major'].str.lower()
df['major'].value_counts().nlargest(n=50)


In [ ]:
#function for cleaning 'major' column
#'na' values changed with 'other' value

def major_simplify(title):
    if 'business management' in str(title).lower():
        return 'management'
    elif 'information technology' in str(title).lower():
        return 'it'
    elif 'math' in str(title).lower():
        return 'mathematics'
    elif 'computer' in str(title).lower():
        return 'it'
    elif 'bio' in str(title).lower():
        return 'biology'
    elif 'tesl' in str(title).lower():
        return 'english'
    elif 'medic' in str(title).lower():
        return 'medicine'
    elif 'account' in str(title).lower():
        return 'accountacy'
    elif 'none' in str(title).lower():
        return np.nan
    elif 'nurs' in str(title).lower():
        return 'nursing'
    elif '-' in str(title).lower():
        return np.nan
    elif 'teach' in str(title).lower():
        return 'teaching'
    elif 'pharma' in str(title).lower():
        return 'pharmacy'
    elif 'no' in str(title).lower():
        return np.nan
    elif 'film' in str(title).lower():
        return 'media'
    elif 'international' in str(title).lower():
        return 'international relations'
    elif 'human' in str(title).lower():
        return 'human resources'
    elif 'art' in str(title).lower():
        return 'arts'
    elif 'islam' in str(title).lower():
        return 'islamic studies'
    elif 'physio' in str(title).lower():
        return 'physiotherapy'
    elif 'socio' in str(title).lower() or 'social' in str(title).lower():
        return 'sociology'
    elif 'bank' in str(title).lower():
        return 'banking'
    elif 'agri' in str(title).lower():
        return 'agriculture'
    elif 'commerce' in str(title).lower() or 'real estate' in str(title).lower():
        return 'marketing'
    elif 'counsel' in str(title).lower():
        return 'counselling'
    elif 'programming' in str(title).lower():
        return 'it'
    elif 'civil' in str(title).lower():
        return 'engineering'
    elif 'ict' in str(title).lower():
        return 'it'
    elif 'communication' in str(title).lower():
        return 'communication'
    elif 'administration' in str(title).lower():
        return 'administration'
    elif 'psycho' in str(title).lower():
        return 'psychology'
    elif 'english' in str(title).lower():
        return 'english'
    elif 'law' in str(title).lower():
        return 'laws'
    elif 'engineering' in str(title).lower():
        return 'engineering'
    elif 'architecture' in str(title).lower():
        return 'architecture'
    elif 'design' in str(title).lower():
        return 'designer'
    else:
        return title

In [ ]:
#call the simplify function

df['major'] = df['major'].apply(major_simplify)

In [ ]:
#Fixing Education and Major column
#The education column has some errors. It should have 1,2,3 and 4; however it has 0's
#Major column has some 0,1 and 2 values

In [ ]:
df['education'].value_counts()

In [ ]:
df[df['education']==0]['major'].notnull().sum()

In [ ]:
#Change Education '0' to value '3' where major is not null
eduzero = df[(df['major'].notnull()) & ((df['education'] == 0))]
eduzero['education'] = 3
df.loc[eduzero.index, 'education'] = eduzero['education']

#changed major '0' to 'no degree' where Education is equal to 0.
#change 0's major to 'no degree', and 0 to 1
majorzero = df[(df['major'].isnull()) & ((df['education'] == 0))]
majorzero['major'] = 'without a degree'
majorzero['education'] = 1
df.loc[majorzero.index, 'major'] = majorzero['major']
df.loc[majorzero.index, 'education'] = majorzero['education']

In [ ]:
#Change education with value 1 to 3 where major is is not null and education is equal to 1
# change 1's to 3 
eduone = df[(df['major'].notnull()) & ((df['education'] == 1))]
eduone['education'] = 3
df.loc[eduone.index, 'education'] = eduone['education']

#Change major with value 1 to 'without a degree' where education is equal to 1
#change 1's major to 'without a degree'
majorone = df[(df['major'].isnull()) & ((df['education'] == 1))]
majorone['major'] = 'without a degree'
df.loc[majorone.index, 'major'] = majorone['major']

In [ ]:
#Change education with value 2 to 3 where major is is not null and education is equal to 2
# change 2's to 3 
edutwo = df[(df['major'].notnull()) & ((df['education'] == 2))]
edutwo['education'] = 3
df.loc[edutwo.index, 'education'] = edutwo['education']

#Change major with value 2 to 'without a degree' where education is equal to 2
#change 2's major to 'no degree'
majortwo = df[(df['major'].isnull()) & ((df['education'] == 2))]
majortwo['major'] = 'without a degree'
df.loc[majortwo.index, 'major'] = majortwo['major']

In [ ]:
df['education'].value_counts()

In [ ]:
df['education'] = df['education'].astype('object')

In [ ]:
df['major'].isna().sum()

In [ ]:
#Major NA values filled with Mode
df['major'] = df.groupby('education')['major'].apply(lambda x: x.fillna(x.mode()[0]))
df['major'].value_counts()

In [ ]:
## changing major names to 'other' if less than 60
major = df['major'].value_counts()
df['major'] = np.where(df['major'].isin(major.index[major < 60]), 'Other', df['major'])

In [ ]:
#replace 'no' to 'without a degree' and 'without a degree' to 'no degree'
df['major'] = df['major'].apply(lambda x: x.replace('no','without a degree'))
df['major'] = df['major'].apply(lambda x: x.replace('without a degree','no degree'))

In [ ]:
df['major']

In [ ]:
#Cleaning Age,it has some unrealistic values
#Some inputs are based on birth year so we are changing those inputs to age

df['age'] = df['age'].apply(lambda x: 2019-x if x > 1798 else x)

#drop age > 100
df.drop(df[df['age'] > 100].index, inplace = True)


In [ ]:
#Some more data cleaning

#replace value of gender 0 with 3
df['gender'].replace({0:3},inplace=True)

#replace value of orientation 0 with 5
df['orientation'].replace({0:5}, inplace=True)

#replace value of religion 0 with 2
df['religion'].replace({0:2}, inplace=True)

#replace value of married 0 with 3 - other
df['married'].replace({0:3}, inplace=True)

#replace value of hand 0 with 3
df['hand'].replace({0:3}, inplace = True)

#Changing zeros to four as a other
df['urban'].replace({0:4}, inplace=True)

df.drop(df[df['familysize'] > 14].index, inplace=True)

#replace value of familysize 0 with 1
df['familysize'].replace({0:1}, inplace=True)

#replace value of voted 0 with 2
df['voted'].replace({0:2}, inplace=True)

# Impute missings in TIPI
imputer = KNNImputer()

##KNNimputer for missing values
df.loc[:,['TIPI_agreeableness','TIPI_extraversion',
          'TIPI_conscientiousness','TIPI_openness_exp','TIPI_emotional_stability']] = imputer.fit_transform(df.loc[:,['TIPI_agreeableness','TIPI_extraversion',
          'TIPI_conscientiousness','TIPI_openness_exp','TIPI_emotional_stability']]) 

##filling null values in country column with mode
df['country'] = df['country'].fillna(df['country'].mode()[0])
df['country'] = df['country'].apply(lambda x: x.replace('NONE','MY'))


In [ ]:
df['country']

In [ ]:
## Changing 'XK' code to 'RS' code because 'XK' code not in 'country_alpha2_to_country_name'
df['country'] = df['country'].apply(lambda x: x.replace('XK','RS'))

##Converting ISO 2 code to ISO 3 code
df['country_alpha_3'] = df['country'].apply(lambda x: country_name_to_country_alpha3(country_alpha2_to_country_name(x)))
df.drop(['country'], axis = 1, inplace=True)

df['country_alpha_3'].value_counts().nlargest(n=20)

In [ ]:
df['country_alpha_3'].value_counts().nlargest(n=20)

In [ ]:
#There were 16 items in VCL, it was asked examinee's whether they know or don't know given vocabulary. 
#3 given words are not actual words which are used as reliability items.
#Hence We created a sumation of VCL; each correct word is +1, and reliability items are -1 each.

In [ ]:
VCL_questions.columns
VCL_positives = ['VCL1', 'VCL2', 'VCL3', 'VCL4', 'VCL5','VCL7', 'VCL8', 'VCL10', 'VCL11', 
                 'VCL13', 'VCL14', 'VCL15', 'VCL16']
VCL_negatives = ['VCL6', 'VCL9', 'VCL12']

df['total_vocabulary'] = np.sum(VCL_questions[VCL_positives], axis=1) - np.sum(VCL_questions[VCL_negatives],axis=1)                                

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df['total']

In [ ]:
plt.figure(figsize=(20,12))
df['major'].value_counts().sort_values(ascending= True).plot(kind='barh', 
                                                             width = 0.9,
                                                             color=list('bgc'))

for i, v in enumerate(df['major'].value_counts().sort_values(ascending= True)):
    plt.text(v + 10, i - 0.45, str(v), color='blue', fontweight='bold')

_ = plt.xlabel("Number of Examinees")
_ = plt.ylabel("Major Names")
_ = plt.title('Total Number of Examinees by Each Major')


In [ ]:
plt.figure(figsize=(20,12))
df['country_alpha_3'].value_counts()[:25].sort_values(ascending= True).plot(kind='barh', 
                                                             width = 0.9,
                                                             color=list('bgc'))

for i, v in enumerate(df['country_alpha_3'].value_counts()[:25].sort_values(ascending= True)):
    plt.text(v + 10, i - 0.15, str(v), color='blue', fontweight='bold')

_ = plt.xlabel("Number of Examinees")
_ = plt.ylabel("Countries")
_ = plt.title('Total Number of Examinees by Each Country')

In [ ]:
# TO PLOT STACKING BARH for DASS ITEMS

# first create a DF with value_counts of each item
counts = []
for col in dass.columns:
  counts.append([dass[col].value_counts()[1], 
                dass[col].value_counts()[2],
                dass[col].value_counts()[3],
                dass[col].value_counts()[4]])
  
#convert list to DF
count = pd.DataFrame(counts, columns =['Did not apply to me at all',
                                       'Applied to me some degree',
                                       'Applied to me considerable degree',
                                       'Applied to me very much'])

count['item'] = dass.columns
# extract numbers in Items column
count['item'] = count.item.str.extract(('(\d+)'))
#save item as INT
count['item'] = count['item'].astype('int32') 



In [ ]:
# Now plot
#create a plot object
_ = plt.style.use('ggplot')

_ = count.sort_values('item', ascending=False).plot(x='item', 
           kind='barh',
           stacked=True,
           width = 0.85, 
           legend = True,
           colormap='Spectral',
           figsize= (12,10)
           )
plt.xlabel("Count")
plt.ylabel("Questions")

plt.legend( bbox_to_anchor = (1.1, -0.1), ncol=len(count.columns))

# add count of each response
for p in _.patches:
    left, bottom, width, height = p.get_bbox().bounds
    if width > 0:
         _.annotate(f'{width:0.0f}', xy=(left+width/2, bottom+height/2), ha='center', va='center',)

The following items were answered as the examinee's felt that way most of the time:

Question 11: I found myself getting upset rather easily.
Question 13: I felt sad and depressed.
Question 18: I felt that I was rather touchy.
Question 34: I felt I was pretty worthless.
Question 40: I was worried about situations in which I might panic and make a fool of myself.
On the other hand, following items were answered as the examinee's felt that way nearly never:

Question 15: I had a feeling of faintness.
Question 23: I had difficulty in swallowing.
It is obvious that most of the participants had mental / emotional symptoms of ADS; and do not have physical symptoms.

In [ ]:
selected_questions = ['Q2E','Q4E','Q7E','Q15E','Q19E','Q23E','Q25E','Q41E']
ps_questions = dict(testelapse[selected_questions])
all_questions = dict(testelapse)

colors = []
for i in all_questions.keys():
    if i in ps_questions.keys():
      colors.append('g')
    else:
      colors.append('b')

In [ ]:
plt.figure(figsize=(20,12))
testelapse[[x for x in testelapse.columns if 'E' in x]].mean().plot(kind='barh', 
                                                                    width = 0.9,
                                                                    color=colors,
                                                                  )
_ = plt.xlabel("Spent Time")
_ = plt.ylabel("Questions")
_ = plt.title("Distribution of Spent Time per Question")

The highest time spent on Question 9, followed by 35 and 25.

Q9 I found myself in situations that made me so anxious I was most relieved when they ended.

Q25 I was aware of the action of my heart in the absence of physical exertion (eg, sense of heart rate increase, heart missing a beat).

Q35 I was intolerant of anything that kept me from getting on with what I was doing.

In [ ]:
import seaborn as sns

In [ ]:
# scores amongst gender, race, religion and orientation
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(25,10))

#ax1 : gender and total score means
_ = sns.barplot( y = df['total'], x = df['gender'], ax= ax1)

ax1.set_title('DASS Mean Scores by Gender')

for i, v in enumerate(df['gender'].value_counts()):
  ax1.text(s = f"n : {v:}", x = i - 0.1, y = 80 , fontsize= 12)
ax1.set_xticklabels(['male','female','other'])   


_ = sns.barplot( y = df['total'], x = df['race'], ax= ax2)

for i, v in enumerate(df['race'].value_counts()):
  ax2.text(s = f"n : \n{v:}", x = i - 0.25, y = 80, fontsize=12)

ax2.set_title('DASS Mean Scores by Race')

#labels for race
labels_race = ['Asian', 
               'Arab',
               'Black', 
               'Indigenous', 
               'Native American', 
               'White', 
               'Other']
_ = ax2.set_xticklabels(labels_race)

_ = plt.figtext(0.5, 0.01,'n defines the total number of participants in each category',
           ha='center', fontsize=12)

Minorities and disadventagenous groups have higher DASS scores. However, group participant distributions in this study are way too much unbalanced. Therefore, it is really hard to make general assumptions rather than having a general idea what is going on.

Male gender obviously have lower DASS scores.

It is also hard to make assumptions about race, however we can have a better idea of asian, arab and black participants. Arabs have the highest DASS score among three groups. Apart from groups; all race groups have DASS scores higher than 100.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,6))
_ = sns.barplot( y = df['total'], x = df['religion'])

religion_labels = ['Agnostic', 
                   'Atheist', 
                   'Buddhist',
                   'Christian (Catholic)',
                   'Christian (Mormon)',
                   'Christian (Protestant)',
                   'Christian (Other)',
                   'Hindu', 
                   'Jewish', 
                   'Muslim', 
                   'Sikh',
                   'Other',
                   ]
plt.title('Total DASS Scores by Religion')
_ = ax.set_xticklabels(religion_labels, rotation= 60)


for i, v in enumerate(df['religion'].value_counts()):
  ax.text(s = f"n : \n{v:}", x = i - 0.30, y = 80, fontsize=12)

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
_ = sns.barplot( y = df['total'], x = df['orientation'])

orientation_labels = ['Heterosexual',
                      'Bisexual',
                      'Homosexual',
                      'Asexual',
                      'Other'
                   ]
plt.title('Total DASS Scores by Orientation')
_ = ax.set_xticklabels(orientation_labels)


for i, v in enumerate(df['orientation'].value_counts()):
  ax.text(s = f"n : {v:}", x = i - 0.30, y = 80, fontsize=12)

Bisexuals have the highest DASS score, and all others apart from Hetero's have higher scores than hetero's.

In [ ]:
bins = [13, 18, 30, 40, 50, 60, 70, 120]
labels = ['13-17','18-29', '30-39', '40-49', '50-59', '60-69', '70+']
df['age_range'] = pd.cut(df.age, bins, labels = labels,include_lowest = True)

df[[x for x in df.columns if '_t' in x] + ['age_range']].groupby('age_range').mean().plot(kind="bar",
                                                                              rot=45,
                                                                              figsize=(20,10),
                                                                              width=0.9,
                                                                              color=list('bgc'))
_ = plt.xlabel("Range of Age")
_ = plt.ylabel("Score")
_ = plt.title("Total Anxiety, Depression, Stress Scores by Age")

In [ ]:
df[[x for x in df.columns if '_p' in x] + ['age_range']].groupby('age_range').mean().plot(kind="bar",
                                                                              rot=45,
                                                                              figsize=(20,10),
                                                                              width=0.9,
                                                                              color=list('bgc'))
_ = plt.xlabel("Range of Age")
_ = plt.ylabel("Score")
_ = plt.title("Distribution of Physical Symptoms by Age")

In [ ]:
df.columns

In [ ]:
df[[x for x in df.columns if '_m' in x] + ['age_range']].groupby('age_range').mean().plot(kind="bar",
                                                                              rot=45,
                                                                              figsize=(20,10),
                                                                              width=0.9,
                                                                              color=list('bgc'))
_ = plt.xlabel("Range of Age")
_ = plt.ylabel("Score")
_ = plt.title("Distribution of Mental Symptoms by Age")

**TIPI**

In [ ]:
tipi_values = df[[i for i in df.columns if 'TIPI' in i]]

fig, ax = plt.subplots(2,3, figsize=(12,8))
_ = plt.style.use('ggplot')


_ = ax[0,0].text(s='TIPI \nSubscale \nScores',
             x= 0.5, y= 0.5,  
             fontsize= 32,
             ha='center', va='center', wrap=True,color = 'darkblue')
#plots

_ = sns.boxplot( y = tipi_values['TIPI_extraversion'], color = 'red',
               ax = ax[0,1])

_ = sns.boxplot( y = tipi_values['TIPI_agreeableness'], color = 'blue',
               ax = ax[0,2])

_ = sns.boxplot( y = tipi_values['TIPI_conscientiousness'], color = 'cyan',
               ax = ax[1,0])

_ = sns.boxplot( y = tipi_values['TIPI_emotional_stability'], color = 'green',
               ax = ax[1,1])

_ = sns.boxplot( y = tipi_values['TIPI_openness_exp'], color = 'yellow',
               ax = ax[1,2])

#titles
_ = ax[0,1].set_title('Extraversion')
_ = ax[0,2].set_title('Agreeableness')
_ = ax[1,0].set_title('Conscientiousness')
_ = ax[1,1].set_title('Emotional Stability')
_ = ax[1,2].set_title('Openness Exp')

In [ ]:
tipi_and_age = df[[x for x in df.columns if 'TIPI' in x] + ['age']]
bins = [13, 18, 30, 40, 50, 60, 70, 120]
labels = ['13-17','18-29', '30-39', '40-49', '50-59', '60-69', '70+']
tipi_and_age['age_range'] = pd.cut(tipi_and_age.age, bins, labels = labels,include_lowest = True)

In [ ]:
fig, ax = plt.subplots(3,2, figsize=(18,16))

_ = ax[0,0].text(s='TIPI \nScores by \nAge',
             x= 0.5, y= 0.5,  
             fontsize= 32,
             ha='center', va='center', wrap=True,color = 'darkblue')

_ = sns.barplot(x = 'age_range', y = 'TIPI_extraversion', data = tipi_and_age, ax= ax[0,1])
_ = sns.barplot(x = 'age_range', y = 'TIPI_agreeableness', data = tipi_and_age, ax= ax[1,0])
_ = sns.barplot(x = 'age_range', y = 'TIPI_conscientiousness', data = tipi_and_age, ax= ax[1,1])
_ = sns.barplot(x = 'age_range', y = 'TIPI_emotional_stability', data = tipi_and_age, ax= ax[2,0])
_ = sns.barplot(x = 'age_range', y = 'TIPI_openness_exp', data = tipi_and_age, ax= ax[2,1])

Majority of the participants are agreeable and conscientious. Also, each personality type score is increasing by age except extraversion.

In [ ]:
def correlation_plot(df, cmap='RdBu_r'):
    fig, ax = plt.subplots(figsize=(12,12))
    corr = df.corr()

    matshow = ax.matshow(corr, cmap=cmap)
    for (x, y), z in np.ndenumerate(corr):
        ax.text(y, x, '{:0.2f}'.format(z), ha='center', va='center',
                bbox=dict(boxstyle='round', facecolor='white', edgecolor='0.3'))

    plt.xticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14, rotation=90)
    plt.yticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14)
    cb = plt.colorbar(matshow, orientation="horizontal")
    ax.tick_params(labelsize=12)
    plt.show()

In [ ]:
personality_values = [x for x in df.columns if 'TIPI' in x]
personalities = df[personality_values]

In [ ]:
df[personality_values]

In [ ]:
df.columns

In [ ]:
dass_headers = ['depression_totalqns','anxiety_totalqns','stress_totalqns']
dass_values = df[dass_headers]

In [ ]:
character = pd.concat([dass_values, personalities], axis=1)
correlation_plot(character, cmap='inferno')